In [43]:
!pip install rdflib
!pip install urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Data obtaining scripts
In below section I implemented list of methods that are used for get data from Internet

In [45]:
import urllib3
import json

urllib3.disable_warnings()

http = urllib3.PoolManager()
application_id = 'a12842cff8d85bdbdb9e779cca01397b'

# Script for obtain all 10 tier tanks info
def get_all_tanks_json():
  r = http.request('GET', f"https://api.tanki.su/wot/encyclopedia/vehicles/?application_id={application_id}&tier=10&fields=is_wheeled%2Ctag%2Ctank_id%2Ctype%2Cdescription%2Cshort_name%2Cnation%2Ctier%2Cname%2Cdefault_profile.hp%2Cdefault_profile.speed_forward%2Cdefault_profile.speed_backward%2Cdefault_profile.engine%2Cdefault_profile.modules%2Cdefault_profile.gun%2Cdefault_profile.turret%2Cdefault_profile.armor%2Cdefault_profile.rapid%2Cdefault_profile.siege")
  data = json.loads(r.data)
  res = data['data']
  return res

# Scripts for obtain player info
def get_user_info_by_nickname(nickname):
  ## get id by nickname
  r = http.request('GET', f"https://api.tanki.su/wot/account/list/?application_id={application_id}&search={nickname}")
  response = json.loads(r.data)
  data = response['data']
  player_id = None
  for player in data:
    if player['nickname'] == nickname:
      player_id = player['account_id']
  
  ## get info by id
  player_info = None
  if player_id != None:
    r = http.request('GET', f"https://api.tanki.su/wot/account/info/?application_id={application_id}&account_id={player_id}&fields=account_id%2Cglobal_rating%2Cclan_id%2Cstatistics.all%2Cstatistics.trees_cut%2Cnickname")
    data = json.loads(r.data)
    if 'data' in data:
      player_info = data['data'][f'{player_id}']

  return player_info

def get_user_info_by_id(player_id):
  ## get info by id
  r = http.request('GET', f"https://api.tanki.su/wot/account/info/?application_id={application_id}&account_id={player_id}&fields=account_id%2Cglobal_rating%2Cclan_id%2Cstatistics.all%2Cstatistics.trees_cut%2Cnickname")
  data = json.loads(r.data)
  player_info = None
  if 'data' in data:
    player_info = data['data'][f'{player_id}']

  return player_info

# Script for obtain stats on each tank by player
def get_player_stats_on_tank_by_id(player_id):
  r = http.request('GET', f"https://api.tanki.su/wot/tanks/stats/?account_id={player_id}&application_id={application_id}&in_garage=1&tank_id=20225%2C3473%2C7169%2C6193%2C15425%2C6145%2C5281%2C13569%2C32001%2C8705%2C2417%2C15953%2C14609%2C16897%2C16913%2C13857%2C3649%2C50849%2C6209%2C34049%2C7249%2C14113%2C9233%2C9297%2C52081%2C5265%2C5425%2C61697%2C4145%2C13825%2C20993%2C14881%2C13905%2C62513%2C4737%2C2721%2C22017%2C17217%2C55841%2C16961%2C58369%2C19489%2C6225%2C21537%2C2977%2C21761%2C8497%2C17729%2C5505%2C12369%2C50689%2C15905%2C13889%2C3681%2C63537%2C12049%2C57937%2C60977%2C6929%2C62257%2C35361%2C19217%2C3937%2C22273%2C11841%2C19473%2C10785%2C9489%2C12305%2C58961%2C8481%2C15697%2C2433%2C2929%2C19985%2C58641%2C5937%2C19969%2C46849%2C19201%2C15617%2C13089%2C19009&fields=account_id%2Cmax_xp%2Call%2Cmax_frags%2Cfrags%2Cmark_of_mastery%2Ctank_id")
  data = json.loads(r.data)
  res = data['data'][f'{player_id}']
  return res

# Scripts for obtain clan stats
def get_clan_info_by_tag(clan_tag):
  ## get id by tag
  r = http.request('GET', f"https://api.tanki.su/wot/clans/list/?application_id={application_id}&search={clan_tag}")
  response = json.loads(r.data)
  data = response['data']
  clan_id = None
  for clan in data:
    if clan['tag'] == clan_tag:
      clan_id = clan['clan_id']
  
  ## get info by id
  clan_info = None
  if clan_id != None:
    r = http.request('GET', f"https://api.tanki.su/wot/clans/info/?application_id={application_id}&clan_id={clan_id}&fields=members_count%2Cdescription%2Ccreator_name%2Cclan_id%2Cleader_name%2Ccreator_id%2Ctag%2Cmembers.account_id%2Cmembers.account_name%2Cold_name%2Cleader_id%2Cmotto%2Cold_tag%2Cname")
    data = json.loads(r.data)
    clan_info = data['data'][f'{clan_id}']

  return clan_info

def get_clan_info_by_id(clan_id):
  ## get info by id
  r = http.request('GET', f"https://api.tanki.su/wot/clans/info/?application_id={application_id}&clan_id={clan_id}&fields=members_count%2Cdescription%2Ccreator_name%2Cclan_id%2Cleader_name%2Ccreator_id%2Ctag%2Cmembers.account_id%2Cmembers.account_name%2Cold_name%2Cleader_id%2Cmotto%2Cold_tag%2Cname")
  data = json.loads(r.data)
  clan_info = data['data'][f'{clan_id}']

  return clan_info

# Script to obtain battles info
def get_battles_info():
  ## Opening JSON file
  f = open('gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/battles-results.json')
  
  ## JSON object as a dictionary
  battles = json.load(f)

  return battles

In [46]:
all_tanks = get_all_tanks_json()
print(f"number of tanks: {len(all_tanks)}")

player_json = get_user_info_by_nickname('MAKIDON__')
print(f"{player_json['nickname']}'s id: {player_json['account_id']}")

player_stats_on_tanks_json = get_player_stats_on_tank_by_id(player_json['account_id'])
print(f"number of tanks that owned by {player_json['nickname']}: {len(player_stats_on_tanks_json)}")

clan_json = get_clan_info_by_tag('NE6P0')
print(f"number of {clan_json['name']} clan members: {len(clan_json['members'])}")

battles_results_json = get_battles_info()
print(f"number of battles: {len(battles_results_json)}")

number of tanks: 83
MAKIDON__'s id: 37657971
number of tanks that owned by MAKIDON__: 1
number of Ne6p0 clan members: 55
number of battles: 7


In [47]:
tank = None
for id, info in all_tanks.items():
  tank = info
  break
  # if info['nation'] == 'czech':
  #   print(f"tank id = {id}; tank name = {info['short_name']}")

In [48]:
# Show tanks info
# all_tanks['20225']

In [49]:
# Create tanks dictionary using tank_name as key
all_tanks_by_name = dict()
for key, info in all_tanks.items():
  all_tanks_by_name[info['name']] = info

In [50]:
# all_tanks_by_name['ИС-7']

In [51]:
# Show common player stats
# player_json

In [52]:
# Show one entity of stats on concrete tank for player
# player_stats_on_tanks_json

In [53]:
# Show clan info
# clan_json

In [54]:
# Show battles info
# battles_results_json

# Filling of ontology

In [55]:
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.serializer import Serializer

ontology_url = 'gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/tanks.owl'

# Create a Graph
tanks_ontology_graph = Graph()

# Parse in an RDF file
tanks_ontology_graph.parse(ontology_url)

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in tanks_ontology_graph:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in tanks_ontology_graph:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"tanks_ontology_graph has {len(tanks_ontology_graph)} statements.")

# print(tanks_ontology_graph.serialize(format="turtle"))

tanks_ontology_graph has 1034 statements.


In [56]:
#Class
Association_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Association')
Battle_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Battle')
BattleResults_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#BattleResults')
Clan_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Clan')
ClanMember_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#ClanMember')
Member_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Member')
Nation_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Nation')
Platoon_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Platoon')
PlatoonMember_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#PlatoonMember')
Player_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Player')
PlayerStatistic_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#PlayerStatistic')
PlayerStatisticOnConcreteTank_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#PlayerStatisticOnConcreteTank')
RapidMode_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#RapidMode')
SiegeMode_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#SiegeMode')
Tank_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Tank')
TankType_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#TankType')
Team_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Team')
TeamMember_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#TeamMember')

#ObjectProperty
hasBattleResults_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasBattleResults')
hasClanMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasClanMember')
hasClanmate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasClanmate')
hasMate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasMate')
hasMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasMember')
hasNation_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasNation')
hasPlatoonMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasPlatoonMember')
hasPlatoonmate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasPlatoonmate')
hasRapidMode_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasRapidMode')
hasResults_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasResults')
hasSiegeMode_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasSiegeMode')
hasStatistic_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasStatistic')
hasTeam_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasTeam')
hasTeamMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasTeamMember')
hasTeammate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasTeammate')
hasType_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasType')
isMemberOf_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOf')
isMemberOfClan_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOfClan')
isMemberOfPlatoon_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOfPlatoon')
isMemberOfTeam_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOfTeam')
isResultsOfBattle_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isResultsOfBattle')
isResultsOfPlayer_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isResultsOfPlayer')
isStatisticOf_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isStatisticOf')
playedOn_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#playedOn')
takePartIn_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#takePartIn')
tankStatisticOf_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tankStatisticOf')
tankStatisticOn_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tankStatisticOn')

#DatatypeProperty
accountId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#accountId')
avgDamageAssisted_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageAssisted')
avgDamageAssistedRadio_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageAssistedRadio')
avgDamageAssistedTrack_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageAssistedTrack')
avgDamageBlocked_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageBlocked')
batResDamage_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResDamage')
batResFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResFrags')
batResPlayerClanTag_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResPlayerClanTag')
batResPlayerName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResPlayerName')
batResTank_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResTank')
batResXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResXp')
batResPlatoonNumber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResPlatoonNumber')
battleAvgXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleAvgXp')
battleId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleId')
battleResultsProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleResultsProperty')
battles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battles')
battlesOnStunningVehicles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battlesOnStunningVehicles')
capturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#capturePoints')
clanId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#clanId')
clanProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#clanProperty')
concreteTankAccountId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankAccountId')
concreteTankAvgDamageBlocked_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankAvgDamageBlocked')
concreteTankBattleAvgXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankBattleAvgXp')
concreteTankBattles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankBattles')
concreteTankBattlesOnStunningVehicles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankBattlesOnStunningVehicles')
concreteTankCapturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankCapturePoints')
concreteTankDamageDealt_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDamageDealt')
concreteTankDamageReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDamageReceived')
concreteTankDirectHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDirectHitsReceived')
concreteTankDraws_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDraws')
concreteTankDroppedCapturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDroppedCapturePoints')
concreteTankExplosionHits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankExplosionHits')
concreteTankExplosionHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankExplosionHitsReceived')
concreteTankFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankFrags')
concreteTankHits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankHits')
concreteTankHitsPercents_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankHitsPercents')
concreteTankLosses_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankLosses')
concreteTankMarkOfMastery_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankMarkOfMastery')
concreteTankMaxFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankMaxFrags')
concreteTankMaxXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankMaxXp')
concreteTankNoDamageDirectHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankNoDamageDirectHitsReceived')
concreteTankPiercings_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankPiercings')
concreteTankPiercingsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankPiercingsReceived')
concreteTankShots_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankShots')
concreteTankSpotted_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankSpotted')
concreteTankStatisticProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankStatisticProperties')
concreteTankStunAssistedDamage_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankStunAssistedDamage')
concreteTankStunNumber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankStunNumber')
concreteTankSurvivedBattles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankSurvivedBattles')
concreteTankTankId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankTankId')
concreteTankTankingFactor_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankTankingFactor')
concreteTankWins_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankWins')
concreteTankXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankXp')
creatorId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#creatorId')
creatorName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#creatorName')
damageDealt_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#damageDealt')
damageReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#damageReceived')
description_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#description')
directHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#directHitsReceived')
draws_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#draws')
droppedCapturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#droppedCapturePoints')
explosionHits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#explosionHits')
explosionHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#explosionHitsReceived')
frags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#frags')
globalRating_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#globalRating')
hits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hits')
hitsPercents_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hitsPercents')
leaderId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#leaderId')
leaderName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#leaderName')
losses_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#losses')
maxDamage_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#maxDamage')
maxFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#maxFrags')
maxXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#maxXp')
membersCount_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#membersCount')
motto_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#motto')
name_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#name')
nickName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#nickName')
noDamageDirectHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#noDamageDirectHitsReceived')
oldName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#oldName')
piercings_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#piercings')
piercingsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#piercingsReceived')
playerProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#playerProperty')
playerStatisticProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#playerStatisticProperties')
rapidModeProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidModeProperties')
rapidSpeedBackward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSpeedBackward')
rapidSpeedForward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSpeedForward')
rapidSuspensionSteeringLockAngle_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSuspensionSteeringLockAngle')
rapidSwitchOffTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSwitchOffTime')
rapidSwitchOnTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSwitchOnTime')
shots_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#shots')
siegeDispersion_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeDispersion')
siegeModeProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeModeProperties')
siegeMoveDownArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeMoveDownArc')
siegeMoveUpArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeMoveUpArc')
siegeReloadTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeReloadTime')
siegeSpeedBackward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSpeedBackward')
siegeSuspensionTraverseSpeed_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSuspensionTraverseSpeed')
siegeSwitchOffTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSwitchOffTime')
siegeSwitchOnTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSwitchOnTime')
siegeAimTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeAimTime')
spotted_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#spotted')
stunNumber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#stunNumber')
survivedBattles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#survivedBattles')
tag_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tag')
tankingFactor_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tankingFactor')
treesCut_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#treesCut')
vechicleArmorHullFront_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorHullFront')
vechicleArmorHullRear_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorHullRear')
vechicleArmorHullSides_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorHullSides')
vechicleArmorTurretFront_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorTurretFront')
vechicleArmorTurretRear_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorTurretRear')
vechicleArmorTurretSides_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorTurretSides')
vechicleDescription_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleDescription')
vechicleFireChance_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleFireChance')
vechicleGunAimTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunAimTime')
vechicleGunCaliber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunCaliber')
vechicleGunDispersion_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunDispersion')
vechicleGunFireRate_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunFireRate')
vechicleGunMoveDownArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunMoveDownArc')
vechicleGunMoveUpArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunMoveUpArc')
vechicleGunReloadTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunReloadTime')
vechicleGunTraverseSpeed_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunTraverseSpeed')
vechicleHp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleHp')
vechicleIsWheeled_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleIsWheeled')
vechicleModulesEngineId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesEngineId')
vechicleModulesGunId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesGunId')
vechicleModulesRadioId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesRadioId')
vechicleModulesSuspensionId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesSuspensionId')
vechicleModulesTurretId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesTurretId')
vechicleName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleName')
vechiclePower_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechiclePower')
vechicleProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleProperty')
vechicleShortName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleShortName')
vechicleSpeedBackward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleSpeedBackward')
vechicleSpeedForward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleSpeedForward')
vechicleTag_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTag')
vechicleTankId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTankId')
vechicleTier_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTier')
vechicleTurretTraverseSpeed_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTurretTraverseSpeed')
vechicleViewRange_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleViewRange')
wins_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#wins')
xp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#xp')
batResPlatoonNumber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResPlatoonNumber')
battleStatus_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleStatus')

#Individual
AT_SPG_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#AT-SPG')
SPG_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#SPG')
heavyTank_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#heavyTank')
lightTank_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#lightTank')
mediumTank_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#mediumTank')
china_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#china')
czech_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#czech')
france_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#france')
germany_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#germany')
italy_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#italy')
japan_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#japan')
poland_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#poland')
sweden_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#sweden')
uk_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#uk')
usa_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#usa')
ussr_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#ussr')

In [57]:
# Dicts for access to tank individual by tank name and id
tanks_ind_dict_by_name = dict()
tanks_ind_dict_by_id = dict()

# Dicts for access to player individual by nickname and id
players_ind_dict_by_name = dict()
players_ind_dict_by_id = dict()

# Dicts for access to clan individual by id
clans_ind_dict_by_id = dict()

In [58]:
# Fill tanks part of graph with data from Internet
## Create tanks triples
for name, info in all_tanks_by_name.items():
  tank_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tank_{info['tag']}")
  tanks_ind_dict_by_name[info['name']] = tank_individual
  tanks_ind_dict_by_id[info['tank_id']] = tank_individual
  tanks_ontology_graph.add((tank_individual, RDF.type, Tank_class))
  # Data props
  tanks_ontology_graph.add((tank_individual, vechicleDescription_dp, Literal(info['description'])))
  tanks_ontology_graph.add((tank_individual, vechicleArmorHullFront_dp, Literal(info['default_profile']['armor']['hull']['front'])))
  tanks_ontology_graph.add((tank_individual, vechicleArmorHullRear_dp, Literal(info['default_profile']['armor']['hull']['rear'])))
  tanks_ontology_graph.add((tank_individual, vechicleArmorHullSides_dp, Literal(info['default_profile']['armor']['hull']['sides'])))
  if info['default_profile']['armor']['turret'] is not None:
    tanks_ontology_graph.add((tank_individual, vechicleArmorTurretFront_dp, Literal(info['default_profile']['armor']['turret']['front'])))
    tanks_ontology_graph.add((tank_individual, vechicleArmorTurretRear_dp, Literal(info['default_profile']['armor']['turret']['rear'])))
    tanks_ontology_graph.add((tank_individual, vechicleArmorTurretSides_dp, Literal(info['default_profile']['armor']['turret']['sides'])))
  tanks_ontology_graph.add((tank_individual, vechicleFireChance_dp, Literal(info['default_profile']['engine']['fire_chance'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunAimTime_dp, Literal(info['default_profile']['gun']['aim_time'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunCaliber_dp, Literal(info['default_profile']['gun']['caliber'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunDispersion_dp, Literal(info['default_profile']['gun']['dispersion'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunFireRate_dp, Literal(info['default_profile']['gun']['fire_rate'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunMoveDownArc_dp, Literal(info['default_profile']['gun']['move_down_arc'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunMoveUpArc_dp, Literal(info['default_profile']['gun']['move_up_arc'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunReloadTime_dp, Literal(info['default_profile']['gun']['reload_time'])))
  tanks_ontology_graph.add((tank_individual, vechicleGunTraverseSpeed_dp, Literal(info['default_profile']['gun']['traverse_speed'])))
  tanks_ontology_graph.add((tank_individual, vechicleHp_dp, Literal(info['default_profile']['hp'])))
  tanks_ontology_graph.add((tank_individual, vechicleIsWheeled_dp, Literal(info['is_wheeled'])))
  tanks_ontology_graph.add((tank_individual, vechicleModulesEngineId_dp, Literal(info['default_profile']['modules']['engine_id'])))
  tanks_ontology_graph.add((tank_individual, vechicleModulesGunId_dp, Literal(info['default_profile']['modules']['gun_id'])))
  tanks_ontology_graph.add((tank_individual, vechicleModulesRadioId_dp, Literal(info['default_profile']['modules']['radio_id'])))
  tanks_ontology_graph.add((tank_individual, vechicleModulesSuspensionId_dp, Literal(info['default_profile']['modules']['suspension_id'])))
  tanks_ontology_graph.add((tank_individual, vechicleModulesTurretId_dp, Literal(info['default_profile']['modules']['turret_id'])))
  tanks_ontology_graph.add((tank_individual, vechicleName_dp, Literal(info['name'])))
  tanks_ontology_graph.add((tank_individual, vechiclePower_dp, Literal(info['default_profile']['engine']['power'])))
  tanks_ontology_graph.add((tank_individual, vechicleShortName_dp, Literal(info['short_name'])))
  tanks_ontology_graph.add((tank_individual, vechicleSpeedBackward_dp, Literal(info['default_profile']['speed_backward'])))
  tanks_ontology_graph.add((tank_individual, vechicleSpeedForward_dp, Literal(info['default_profile']['speed_forward'])))
  tanks_ontology_graph.add((tank_individual, vechicleTag_dp, Literal(info['tag'])))
  tanks_ontology_graph.add((tank_individual, vechicleTankId_dp, Literal(info['tank_id'])))
  tanks_ontology_graph.add((tank_individual, vechicleTier_dp, Literal(info['tier'])))
  tanks_ontology_graph.add((tank_individual, vechicleTurretTraverseSpeed_dp, Literal(info['default_profile']['turret']['traverse_speed'])))
  tanks_ontology_graph.add((tank_individual, vechicleViewRange_dp, Literal(info['default_profile']['turret']['view_range'])))
  # Object props
  ## Type
  if info['type'] == 'AT-SPG':
    tanks_ontology_graph.add((tank_individual, hasType_op, AT_SPG_ind))
  elif info['type'] == 'SPG':
    tanks_ontology_graph.add((tank_individual, hasType_op, SPG_ind))
  elif info['type'] == 'mediumTank':
    tanks_ontology_graph.add((tank_individual, hasType_op, mediumTank_ind))
  elif info['type'] == 'lightTank':
    tanks_ontology_graph.add((tank_individual, hasType_op, lightTank_ind))
  elif info['type'] == 'heavyTank':
    tanks_ontology_graph.add((tank_individual, hasType_op, heavyTank_ind))
  ## Nation
  tanks_ontology_graph.add((tank_individual, hasNation_op, URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#{info['nation']}")))
  ## Siege mode
  if info['default_profile']['siege'] is not None:
    siege_mode = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeMode_{info['tag']}")
    tanks_ontology_graph.add((siege_mode, RDF.type, SiegeMode_class))
    tanks_ontology_graph.add((siege_mode, siegeDispersion_dp, Literal(info['default_profile']['siege']['dispersion'])))
    tanks_ontology_graph.add((siege_mode, siegeMoveDownArc_dp, Literal(info['default_profile']['siege']['move_down_arc'])))
    tanks_ontology_graph.add((siege_mode, siegeMoveUpArc_dp, Literal(info['default_profile']['siege']['move_up_arc'])))
    tanks_ontology_graph.add((siege_mode, siegeReloadTime_dp, Literal(info['default_profile']['siege']['reload_time'])))
    tanks_ontology_graph.add((siege_mode, siegeSpeedBackward_dp, Literal(info['default_profile']['siege']['speed_backward'])))
    tanks_ontology_graph.add((siege_mode, siegeSuspensionTraverseSpeed_dp, Literal(info['default_profile']['siege']['suspension_traverse_speed'])))
    tanks_ontology_graph.add((siege_mode, siegeSwitchOffTime_dp, Literal(info['default_profile']['siege']['switch_off_time'])))
    tanks_ontology_graph.add((siege_mode, siegeSwitchOnTime_dp, Literal(info['default_profile']['siege']['switch_on_time'])))
    tanks_ontology_graph.add((siege_mode, siegeAimTime_dp, Literal(info['default_profile']['siege']['aim_time'])))
    tanks_ontology_graph.add((tank_individual, hasSiegeMode_op, siege_mode))
  ## Rapid mode
  if info['default_profile']['rapid'] is not None:
    rapid_mode = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidMode_{info['tag']}")
    tanks_ontology_graph.add((rapid_mode, RDF.type, RapidMode_class))
    tanks_ontology_graph.add((rapid_mode, rapidSpeedBackward_dp, Literal(info['default_profile']['rapid']['speed_backward'])))
    tanks_ontology_graph.add((rapid_mode, rapidSpeedForward_dp, Literal(info['default_profile']['rapid']['speed_forward'])))
    tanks_ontology_graph.add((rapid_mode, rapidSuspensionSteeringLockAngle_dp, Literal(info['default_profile']['rapid']['suspension_steering_lock_angle'])))
    tanks_ontology_graph.add((rapid_mode, rapidSwitchOffTime_dp, Literal(info['default_profile']['rapid']['switch_off_time'])))
    tanks_ontology_graph.add((rapid_mode, rapidSwitchOnTime_dp, Literal(info['default_profile']['rapid']['switch_on_time'])))
    tanks_ontology_graph.add((tank_individual, hasRapidMode_op, rapid_mode))

In [59]:
# Get players nicnknames with data from battles file
for battle in battles_results_json:
  for player in battle['ally_team_results']:
    player_ind = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#player_{player['nickname']}")
    players_ind_dict_by_name[player['nickname']] = player_ind
    tanks_ontology_graph.add((player_ind, RDF.type, Player_class))
    tanks_ontology_graph.add((player_ind, nickName_dp, Literal(player['nickname'])))

  for player in battle['enemy_team_results']:
    player_ind = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#player_{player['nickname']}")
    players_ind_dict_by_name[player['nickname']] = player_ind
    tanks_ontology_graph.add((player_ind, RDF.type, Player_class))
    tanks_ontology_graph.add((player_ind, nickName_dp, Literal(player['nickname'])))

print(len(players_ind_dict_by_name))

210


In [60]:
def add_player_to_graph(player_info, nickname, player_individual):
    ## Fill player info part
    account_id = player_info['account_id']
    print(f"\tprocessing info about account id:{account_id}")
    players_ind_dict_by_id[account_id] = players_ind_dict_by_name[nickname]

    ### Common
    tanks_ontology_graph.add((player_individual, accountId_dp, Literal(player_info['account_id'])))
    tanks_ontology_graph.add((player_individual, clanId_dp, Literal(player_info['clan_id'])))

    ### Statistic
    player_statistic_ind = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#player_stats_{account_id}")
    tanks_ontology_graph.add((player_statistic_ind, RDF.type, PlayerStatistic_class))
    tanks_ontology_graph.add((player_statistic_ind, isStatisticOf_op, player_individual))
    tanks_ontology_graph.add((player_individual, hasStatistic_op, player_statistic_ind))
    tanks_ontology_graph.add((player_statistic_ind, avgDamageAssisted_dp, Literal(player_info['statistics']['all']['avg_damage_assisted'])))
    tanks_ontology_graph.add((player_statistic_ind, avgDamageAssistedRadio_dp, Literal(player_info['statistics']['all']['avg_damage_assisted_radio'])))
    tanks_ontology_graph.add((player_statistic_ind, explosionHits_dp, Literal(player_info['statistics']['all']['explosion_hits'])))
    tanks_ontology_graph.add((player_statistic_ind, damageReceived_dp, Literal(player_info['statistics']['all']['damage_received'])))
    tanks_ontology_graph.add((player_statistic_ind, droppedCapturePoints_dp, Literal(player_info['statistics']['all']['dropped_capture_points'])))
    tanks_ontology_graph.add((player_statistic_ind, frags_dp, Literal(player_info['statistics']['all']['frags'])))
    tanks_ontology_graph.add((player_statistic_ind, battles_dp, Literal(player_info['statistics']['all']['battles'])))
    tanks_ontology_graph.add((player_statistic_ind, battleAvgXp_dp, Literal(player_info['statistics']['all']['battle_avg_xp'])))
    tanks_ontology_graph.add((player_statistic_ind, avgDamageBlocked_dp, Literal(player_info['statistics']['all']['avg_damage_blocked'])))
    tanks_ontology_graph.add((player_statistic_ind, directHitsReceived_dp, Literal(player_info['statistics']['all']['direct_hits_received'])))
    tanks_ontology_graph.add((player_statistic_ind, avgDamageAssistedTrack_dp, Literal(player_info['statistics']['all']['avg_damage_assisted_track'])))
    tanks_ontology_graph.add((player_statistic_ind, globalRating_dp, Literal(player_info['global_rating'])))
    tanks_ontology_graph.add((player_statistic_ind, battlesOnStunningVehicles_dp, Literal(player_info['statistics']['all']['battles_on_stunning_vehicles'])))
    tanks_ontology_graph.add((player_statistic_ind, hits_dp, Literal(player_info['statistics']['all']['hits'])))
    tanks_ontology_graph.add((player_statistic_ind, capturePoints_dp, Literal(player_info['statistics']['all']['capture_points'])))
    tanks_ontology_graph.add((player_statistic_ind, damageDealt_dp, Literal(player_info['statistics']['all']['damage_dealt'])))
    tanks_ontology_graph.add((player_statistic_ind, draws_dp, Literal(player_info['statistics']['all']['draws'])))
    tanks_ontology_graph.add((player_statistic_ind, explosionHitsReceived_dp, Literal(player_info['statistics']['all']['explosion_hits_received'])))
    tanks_ontology_graph.add((player_statistic_ind, piercings_dp, Literal(player_info['statistics']['all']['piercings'])))
    tanks_ontology_graph.add((player_statistic_ind, maxFrags_dp, Literal(player_info['statistics']['all']['max_frags'])))
    tanks_ontology_graph.add((player_statistic_ind, tankingFactor_dp, Literal(player_info['statistics']['all']['tanking_factor'])))
    tanks_ontology_graph.add((player_statistic_ind, maxDamage_dp, Literal(player_info['statistics']['all']['max_damage'])))
    tanks_ontology_graph.add((player_statistic_ind, spotted_dp, Literal(player_info['statistics']['all']['spotted'])))
    tanks_ontology_graph.add((player_statistic_ind, maxXp_dp, Literal(player_info['statistics']['all']['max_xp'])))
    tanks_ontology_graph.add((player_statistic_ind, xp_dp, Literal(player_info['statistics']['all']['xp'])))
    tanks_ontology_graph.add((player_statistic_ind, shots_dp, Literal(player_info['statistics']['all']['shots'])))
    tanks_ontology_graph.add((player_statistic_ind, piercingsReceived_dp, Literal(player_info['statistics']['all']['piercings_received'])))
    tanks_ontology_graph.add((player_statistic_ind, noDamageDirectHitsReceived_dp, Literal(player_info['statistics']['all']['no_damage_direct_hits_received'])))
    tanks_ontology_graph.add((player_statistic_ind, losses_dp, Literal(player_info['statistics']['all']['losses'])))
    tanks_ontology_graph.add((player_statistic_ind, survivedBattles_dp, Literal(player_info['statistics']['all']['survived_battles'])))
    tanks_ontology_graph.add((player_statistic_ind, hitsPercents_dp, Literal(player_info['statistics']['all']['hits_percents'])))
    tanks_ontology_graph.add((player_statistic_ind, wins_dp, Literal(player_info['statistics']['all']['wins'])))
    tanks_ontology_graph.add((player_statistic_ind, stunNumber_dp, Literal(player_info['statistics']['all']['stun_number'])))
    tanks_ontology_graph.add((player_statistic_ind, treesCut_dp, Literal(player_info['statistics']['trees_cut'])))
    
    ### Statistic on conrete tanks
    player_stats_on_tanks_json = get_player_stats_on_tank_by_id(account_id)
    if player_stats_on_tanks_json is not None:
      print(f"\t\tplayer №{executed_player_no} has {len(player_stats_on_tanks_json)} tanks")
      for concrete_tank_stats in player_stats_on_tanks_json:
        player_stats_on_tank_ind = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#player_{account_id}_stats_on_{concrete_tank_stats['tank_id']}")
        tanks_ontology_graph.add((player_stats_on_tank_ind, RDF.type, PlayerStatisticOnConcreteTank_class))
        tanks_ontology_graph.add((player_stats_on_tank_ind, tankStatisticOf_op, player_individual))
        tanks_ontology_graph.add((player_stats_on_tank_ind, tankStatisticOn_op, tanks_ind_dict_by_id[concrete_tank_stats['tank_id']]))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankSpotted_dp, Literal(concrete_tank_stats['all']['spotted'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankBattlesOnStunningVehicles_dp, Literal(concrete_tank_stats['all']['battles_on_stunning_vehicles'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankAvgDamageBlocked_dp, Literal(concrete_tank_stats['all']['avg_damage_blocked'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankCapturePoints_dp, Literal(concrete_tank_stats['all']['capture_points'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankExplosionHits_dp, Literal(concrete_tank_stats['all']['explosion_hits'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankPiercings_dp, Literal(concrete_tank_stats['all']['piercings'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankXp_dp, Literal(concrete_tank_stats['all']['xp'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankSurvivedBattles_dp, Literal(concrete_tank_stats['all']['survived_battles'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankDroppedCapturePoints_dp, Literal(concrete_tank_stats['all']['dropped_capture_points'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankDamageDealt_dp, Literal(concrete_tank_stats['all']['damage_dealt'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankHitsPercents_dp, Literal(concrete_tank_stats['all']['hits_percents'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankDraws_dp, Literal(concrete_tank_stats['all']['draws'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankBattles_dp, Literal(concrete_tank_stats['all']['battles'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankDamageReceived_dp, Literal(concrete_tank_stats['all']['damage_received'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankFrags_dp, Literal(concrete_tank_stats['all']['frags'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankStunNumber_dp, Literal(concrete_tank_stats['all']['stun_number'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankDirectHitsReceived_dp, Literal(concrete_tank_stats['all']['direct_hits_received'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankStunAssistedDamage_dp, Literal(concrete_tank_stats['all']['stun_assisted_damage'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankHits_dp, Literal(concrete_tank_stats['all']['hits'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankBattleAvgXp_dp, Literal(concrete_tank_stats['all']['battle_avg_xp'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankWins_dp, Literal(concrete_tank_stats['all']['wins'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankLosses_dp, Literal(concrete_tank_stats['all']['losses'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankPiercingsReceived_dp, Literal(concrete_tank_stats['all']['piercings_received'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankNoDamageDirectHitsReceived_dp, Literal(concrete_tank_stats['all']['no_damage_direct_hits_received'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankShots_dp, Literal(concrete_tank_stats['all']['shots'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankExplosionHitsReceived_dp, Literal(concrete_tank_stats['all']['explosion_hits_received'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankTankingFactor_dp, Literal(concrete_tank_stats['all']['tanking_factor'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankAccountId_dp, Literal(concrete_tank_stats['account_id'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankMaxXp_dp, Literal(concrete_tank_stats['max_xp'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankMaxFrags_dp, Literal(concrete_tank_stats['max_frags'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankMarkOfMastery_dp, Literal(concrete_tank_stats['mark_of_mastery'])))
        tanks_ontology_graph.add((player_stats_on_tank_ind, concreteTankTankId_dp, Literal(concrete_tank_stats['tank_id'])))

In [ ]:
# Fill graph with players info
executed_player_no = 0
for nickname, player_individual in players_ind_dict_by_name.items():
  executed_player_no += 1
  print(f"execute player №{executed_player_no}")

  player_info = get_user_info_by_nickname(nickname)
  if player_info is not None:
    clan_id = player_info['clan_id']
    ## Add clan to graph for further extending it
    if clan_id is not None:
      print(f"\t\tplayer №{executed_player_no} has clan")
      clan_ind = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#clan_{clan_id}")
      clans_ind_dict_by_id[clan_id] = clan_ind
      tanks_ontology_graph.add((clan_ind, RDF.type, Clan_class))
      tanks_ontology_graph.add((clan_ind, clanId_dp, Literal(clan_ind)))

    ## Add player to graph
    add_player_to_graph(player_info, nickname, player_individual)

In [ ]:
# Fill clans
for clan_id, clan_individual in clans_ind_dict_by_id.items():
  clan_info = get_clan_info_by_id(clan_id)
  ## Fill data props
  tanks_ontology_graph.add((clan_individual, leaderId_dp, Literal(clan_info['leader_id'])))
  tanks_ontology_graph.add((clan_individual, tag_dp, Literal(clan_info['tag'])))
  tanks_ontology_graph.add((clan_individual, membersCount_dp, Literal(clan_info['members_count'])))
  tanks_ontology_graph.add((clan_individual, leaderName_dp, Literal(clan_info['leader_name'])))
  tanks_ontology_graph.add((clan_individual, description_dp, Literal(clan_info['description'])))
  tanks_ontology_graph.add((clan_individual, oldName_dp, Literal(clan_info['old_name'])))
  tanks_ontology_graph.add((clan_individual, motto_dp, Literal(clan_info['motto'])))
  tanks_ontology_graph.add((clan_individual, name_dp, Literal(clan_info['name'])))
  tanks_ontology_graph.add((clan_individual, creatorName_dp, Literal(clan_info['creator_name'])))
  tanks_ontology_graph.add((clan_individual, creatorId_dp, Literal(clan_info['creator_id'])))
  
  ## Create members individuals
  clan_members = []
  clan_info_members = []
  # if (len(clan_info['members']) >= 5):
  #   clan_info_members = clan_info['members'][:5]
  # else:
  #   clan_info_members = clan_info['members']
  clan_info_members = clan_info['members']

  for member in clan_info_members:
    executed_player_no += 1
    nickname = member['account_name']
    player_id = member['account_id']
    ## fill graph with new players
    if player_id not in players_ind_dict_by_id:
      player_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#player_{nickname}")
      players_ind_dict_by_name[nickname] = player_individual
      players_ind_dict_by_name[player['nickname']] = player_ind
      tanks_ontology_graph.add((player_ind, RDF.type, Player_class))
      tanks_ontology_graph.add((player_ind, nickName_dp, Literal(player['nickname'])))
      player_info = get_user_info_by_id(player_id)
      if player_info is not None:
        add_player_to_graph(player_info, nickname, player_individual)

    if player_id in players_ind_dict_by_id.keys():
      member_individual = players_ind_dict_by_id[player_id]
      tanks_ontology_graph.add((member_individual, RDF.type, ClanMember_class))
      tanks_ontology_graph.add((member_individual, isMemberOfClan_op, clan_individual))
      tanks_ontology_graph.add((clan_individual, hasClanMember_op, member_individual))
      for clan_member in clan_members:
        tanks_ontology_graph.add((member_individual, hasClanmate_op, clan_member))
      clan_members.append(member_individual)

In [67]:
len(tanks_ontology_graph)

4595028

In [68]:
players_ind_dict_by_name

# Fill battles
for battle in battles_results_json:
  battle_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battle_{battle['id']}")
  ally_team_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#ally_team_battle_{battle['id']}")
  enemy_team_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#enemy_team_battle_{battle['id']}")
  tanks_ontology_graph.add((battle_individual, RDF.type, Battle_class))
  tanks_ontology_graph.add((ally_team_individual, RDF.type, Team_class))
  tanks_ontology_graph.add((enemy_team_individual, RDF.type, Team_class))
  tanks_ontology_graph.add((battle_individual, battleStatus_dp, Literal(battle['status'])))
  tanks_ontology_graph.add((battle_individual, hasTeam_op, ally_team_individual))
  tanks_ontology_graph.add((battle_individual, hasTeam_op, enemy_team_individual))
  tanks_ontology_graph.add((ally_team_individual, takePartIn_op, battle_individual))
  tanks_ontology_graph.add((enemy_team_individual, takePartIn_op, battle_individual))

  allies = []
  allies_platoons = dict()
  for player_results in battle['ally_team_results']:
    player_ind = players_ind_dict_by_name[player_results['nickname']]
    tank_ind = tanks_ind_dict_by_name[player_results['tank_name']]
    battle_results_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battle_{battle['id']}_results_of_{player_results['nickname']}")

    tanks_ontology_graph.add((player_ind, RDF.type, TeamMember_class))
    tanks_ontology_graph.add((battle_results_individual, RDF.type, BattleResults_class))
    tanks_ontology_graph.add((battle_results_individual, isResultsOfPlayer_op, player_ind))
    tanks_ontology_graph.add((battle_results_individual, isResultsOfBattle_op, battle_individual))
    tanks_ontology_graph.add((battle_results_individual, playedOn_op, tank_ind))
    tanks_ontology_graph.add((player_ind, takePartIn_op, ally_team_individual))
    tanks_ontology_graph.add((ally_team_individual, hasTeamMember_op, player_ind))
    tanks_ontology_graph.add((battle_results_individual, batResDamage_dp, Literal(player_results['damage'])))
    tanks_ontology_graph.add((battle_results_individual, batResFrags_dp, Literal(player_results['frags'])))
    tanks_ontology_graph.add((battle_results_individual, batResPlayerClanTag_dp, Literal(player_results['clan_tag'])))
    tanks_ontology_graph.add((battle_results_individual, batResPlayerName_dp, Literal(player_results['nickname'])))
    tanks_ontology_graph.add((battle_results_individual, batResTank_dp, Literal(player_results['tank_name'])))
    tanks_ontology_graph.add((battle_results_individual, batResXp_dp, Literal(player_results['xp'])))
    tanks_ontology_graph.add((battle_results_individual, batResPlatoonNumber_dp, Literal(player_results['platoon_number'])))

    for ally in allies:
      tanks_ontology_graph.add((ally, hasTeammate_op, player_ind))
    allies.append(player_ind)

    if player_results['platoon_number'] != "":
      if player_results['platoon_number'] not in allies_platoons:
        platoon_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#platoon_ally_{player_results['platoon_number']}_of_battle{battle['id']}")
        tanks_ontology_graph.add((platoon_individual, RDF.type, Platoon_class))
        tanks_ontology_graph.add((platoon_individual, takePartIn_op, ally_team_individual))
        allies_platoons[player_results['platoon_number']] = platoon_individual
      tanks_ontology_graph.add((player_ind, RDF.type, PlatoonMember_class))  
      platoon_ind = allies_platoons[player_results['platoon_number']]
      tanks_ontology_graph.add((platoon_ind, hasPlatoonMember_op, player_ind))
      tanks_ontology_graph.add((player_ind, isMemberOfPlatoon_op, platoon_ind))

  enemys = []
  enemys_platoons = dict()
  for player_results in battle['enemy_team_results']:
    player_ind = players_ind_dict_by_name[player_results['nickname']]
    tank_ind = tanks_ind_dict_by_name[player_results['tank_name']]
    battle_results_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battle_{battle['id']}_results_of_{player_results['nickname']}")

    tanks_ontology_graph.add((player_ind, RDF.type, TeamMember_class))
    tanks_ontology_graph.add((battle_results_individual, RDF.type, BattleResults_class))
    tanks_ontology_graph.add((battle_results_individual, isResultsOfPlayer_op, player_ind))
    tanks_ontology_graph.add((battle_results_individual, isResultsOfBattle_op, battle_individual))
    tanks_ontology_graph.add((battle_results_individual, playedOn_op, tank_ind))
    tanks_ontology_graph.add((player_ind, takePartIn_op, enemy_team_individual))
    tanks_ontology_graph.add((enemy_team_individual, hasTeamMember_op, player_ind))
    tanks_ontology_graph.add((battle_results_individual, batResDamage_dp, Literal(player_results['damage'])))
    tanks_ontology_graph.add((battle_results_individual, batResFrags_dp, Literal(player_results['frags'])))
    tanks_ontology_graph.add((battle_results_individual, batResPlayerClanTag_dp, Literal(player_results['clan_tag'])))
    tanks_ontology_graph.add((battle_results_individual, batResPlayerName_dp, Literal(player_results['nickname'])))
    tanks_ontology_graph.add((battle_results_individual, batResTank_dp, Literal(player_results['tank_name'])))
    tanks_ontology_graph.add((battle_results_individual, batResXp_dp, Literal(player_results['xp'])))
    tanks_ontology_graph.add((battle_results_individual, batResPlatoonNumber_dp, Literal(player_results['platoon_number'])))

    for enemy in enemys:
      tanks_ontology_graph.add((enemy, hasTeammate_op, player_ind))
    enemys.append(player_ind)

    if player_results['platoon_number'] != "":
      if player_results['platoon_number'] not in enemys_platoons:
        platoon_individual = URIRef(f"http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#platoon_enemy_{player_results['platoon_number']}_of_battle{battle['id']}")
        tanks_ontology_graph.add((platoon_individual, RDF.type, Platoon_class))
        tanks_ontology_graph.add((platoon_individual, takePartIn_op, enemy_team_individual))
        enemys_platoons[player_results['platoon_number']] = platoon_individual
      tanks_ontology_graph.add((player_ind, RDF.type, PlatoonMember_class))
      platoon_ind = enemys_platoons[player_results['platoon_number']]
      tanks_ontology_graph.add((platoon_ind, hasPlatoonMember_op, player_ind))
      tanks_ontology_graph.add((player_ind, isMemberOfPlatoon_op, platoon_ind))
    

print(len(players_ind_dict_by_name))

6834


In [69]:
# Save the entire Graph in the RDF Turtle format
tanks_ontology_graph.serialize('gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/filled-tanks-ontology.owl', format='turtle')

<Graph identifier=Nbc058687766c407c916c69581bdf03c4 (<class 'rdflib.graph.Graph'>)>